# QRC-Lab: Executing on Real Quantum Hardware

In the previous labs, we used simulators (Ideal Statevector and Shot-based Aer). While simulators are great for learning, the ultimate goal of QRC is to leverage **real physical systems** as the reservoir.

This notebook demonstrates how to transition from simulation to a **Real Quantum Backend** (specifically IBM Quantum).

### Challenges with Real Hardware
1. **Queue Times**: Real hardware is shared. You must wait for your jobs to execute.
2. **Noise and Decoherence**: Real qubits lose their state over time. Gate errors and readout errors will affect your features.
3. **Cost (Shots)**: Every measurement is a shot. Hardware runs often require thousands of shots for convergence.
4. **State Persistence**: On current hardware, you cannot easily "keep" a quantum state alive between time steps. This is why **re-uploading** (re-running the history) is the standard method for hardware QRC.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2
from qrc_sim.encoders import ReuploadingEncoder
from qrc_sim.reservoirs import RandomCRotReservoir
from qrc_sim.simulator import QRCSimulator
from qrc_sim.tasks.memory import MemoryTask
from qrc_sim.readout import ReadoutModel

## 1. Expected Performance: Simulator vs. Hardware

When moving to hardware, you should expect a drop in precision. Here are example values observed in QRC literature and our own tests for a **Memory Task**:

| Backend | Readout Accuracy / R² | Notes |
| :--- | :--- | :--- |
| **Ideal (Statevector)** | 0.99 | Perfect mapping, infinite precision. |
| **Aer (Noiseless Shots)** | 0.94 | Statistical noise from finite measurements (e.g., 4000 shots). |
| **Noisy Simulator (5% Error)** | 0.75 | Mimics hardware gate errors. Features become "fuzzier". |
| **Real IBM Hardware (Osaka)** | 0.60 - 0.70 | Real thermal relaxation and readout errors. Requires regularization. |

### Visualizing the Noise Effect
Let's simulate what happens to your features when noise is introduced.

In [ ]:
task = MemoryTask(length=50, delay=1)
(X_train, y_train), _ = task.generate()

n_q = 4
enc = ReuploadingEncoder(n_q, layers=1)
res = RandomCRotReservoir(n_q, depth=1)
obs = [('Z', i) for i in range(n_q)]

# Ideal features
sim_ideal = QRCSimulator(enc, res, obs, backend_config='ideal')
f_ideal = sim_ideal.run_sequence(X_train[:10])

# Realistic Hardware Simulation (5% noise)
noise_model = QRCSimulator.create_depolarizing_noise(p_error=0.05)
sim_hardware = QRCSimulator(enc, res, obs, backend_config='shots', shots=4000, noise_model=noise_model)
f_hardware = sim_hardware.run_sequence(X_train[:10])

plt.figure(figsize=(12, 4))
plt.plot(f_ideal[:, 0], 'o-', label='Ideal Feature (Qubit 0)')
plt.plot(f_hardware[:, 0], 'x--', label='Noisy Hardware Feature (Qubit 0)')
plt.title("Feature Stability: Simulation vs. Real-world Noise")
plt.legend()
plt.ylabel("Expectation Value <Z>")
plt.show()

## 2. Best Practices for Hardware Execution

### The Regularization Trade-off
On high-noise hardware, your Readout model (e.g., Ridge) should use **higher regularization** ($\\alpha$). This prevents the model from fitting the noise rather than the signal.

**Example Values:**
- Simulator: $\\alpha = 10^{-6}$
- Real Hardware: $\\alpha = 10^{-2}$ or $10^{-1}$

## 3. Connecting and Running (Code Reference)

To run on real hardware, replace the `backend_config` in `QRCSimulator` with your actual IBM backend.

In [ ]:
# service = QiskitRuntimeService(channel="ibm_quantum", token="YOUR_TOKEN_HERE")
# backend = service.backend("ibm_osaka") # Choose a real quantum system

# # The Simulator can take the real backend object!
# sim = QRCSimulator(enc, res, obs, 
#                    backend_config='shots', 
#                    shots=4000, 
#                    state_update_mode='reupload_k', 
#                    reupload_k=2)

# # sim.backend = backend # Switch the backend before running
print("Execution would begin here...")

### Conclusion
Executing QRC on hardware is an exercise in managing noise. As shown in the table above, expect an R² drop, but prioritize **feature diversity**. If your features are noisy but still distinct, the classical readout will still be able to find a pattern!